# 要約 
このJupyter Notebookは、KerasおよびKerasNLPを使用して、LMSYS - Chatbot Arenaコンペティションにおける人間による好み予測のためのモデルを構築するスターターノートブックです。具体的には、LLM（大規模言語モデル）によって生成されたチャットボットの応答に対して、どの応答がユーザーに好まれるかを予測する問題に取り組んでいます。

## 問題
コンペティションの目的は、ユーザーが提示するプロンプトに対して複数の応答から選んだ場合に、その選ばれる可能性の高い応答を予測することです。これは、多数のLLMからの応答に基づいて行われます。

## 使用している手法
このノートブックでは、**DebertaV3**というモデルをファインチューニングし、選択肢問題の形式で応答を処理しています。具体的には、各プロンプトと応答をペアで使用し、LLMがどの応答を好ましく評価するかを学習します。モデルにはKerasNLPを使用し、混合精度トレーニングを利用することでトレーニング時間を短縮し、GPUのメモリ使用量を減らしています。

## 使用しているライブラリ
- **Keras**: モデルの構築とトレーニングに使用され、ユーザーが指定したバックエンド（TensorFlow、PyTorch、JAX）を選択して使用できます。
- **KerasNLP**: NLPのために設計されたKerasのサブライブラリで、特にDebertaV3のような様々な事前学習モデルの実装を提供しています。
- **TensorFlow**: モデルのトレーニングと評価を行うためのエコシステム。
- **NumPy/Pandas**: データ操作および分析に使用。
- **Matplotlib/Plotly**: データの可視化に使用。

## その他
ノートブックは、データの読み込みから前処理、モデル構築、訓練、予測、提出ファイルの作成までのプロセスが詳細に示されており、各ステップでのコードが含まれています。多段階のアプローチを取り入れており、最良のモデル評価を行うためのチェックポイントも設定されています。また、今後の改善ポイントとして、大規模モデルへの拡大、クロスバリデーション、応答の順序シャッフル、エポック数の増加などが提案されています。

---


# 用語概説 
以下は、Jupyter Notebookに関連する専門用語の簡単な解説です。特に、初心者がつまずきやすい詳細や実務経験がないと馴染みが薄いものに焦点を当てています。

### 専門用語の解説

1. **ファインチューニング (Fine-tuning)**:
   事前に学習したモデルに新たなデータを用いて追加の学習を行うこと。特に転移学習の文脈で、新しいタスクに特化させるために用いられる。

2. **混合精度 (Mixed Precision)**:
   トレーニング時に異なる精度のデータ型を混ぜて使用する技術。通常、計算処理の効率を高め、GPUメモリの使用量を削減するために使用される。たとえば、`float16`（半精度）と`float32`（単精度）を組み合わせることが一般的。

3. **前処理器 (Preprocessor)**:
   生データを機械学習モデルの入力として使用できる形式に変換するためのツール。具体的には、トークン化、正規化、パディング等が含まれる。

4. **トークン化 (Tokenization)**:
   生のテキストデータを意味のある単位（トークン）に分割するプロセス。これにより、テキストをモデルが理解しやすい形式に変換できる。

5. **パディング (Padding)**:
   シーケンスの長さを一定にするために、短いシーケンスに追加のトークン（通常はゼロ）を付与すること。これにより、ミニバッチ処理を効率的に行うことが可能になる。

6. **エンコーディング (Encoding)**:
   文字列データを数値データに変換するプロセス。特に、モデルがテキストを処理できる形式に変換するために重要で、UTF-8などの標準的な文字エンコーディングが使われる。

7. **層別化 (Stratification)**:
   データを分割する際に、特定の属性に基づいてその分布を保つようにすること。特に、異なるクラスのデータサンプルが均等に分割されるようにするために使用される。

8. **ロス (Loss)**:
   モデルの予測が正解とどれだけ乖離しているかを示す指標。学習中はこの値を最小化することを目的とする。`ログロス`は特に、クラス確率の予測精度を測るために用いられる。

9. **ウェイトシェアリング (Weight Sharing)**:
   複数の部分で同一の重みを共有すること。特に、同じ入力形式の異なる部分（例えば、異なる応答の処理）に同じモデルのパラメータを使用する手法。

10. **メトリック (Metric)**:
    モデルのパフォーマンスを評価するために使用される指標。精度やロスなど、モデルの性能を測るために選択される。

11. **入力層 (Input Layer)**:
    モデルにデータを供給するための層で、データの特徴を定義します。形状やデータ型が指定されることが一般的です。

12. **グローバル平均プーリング (Global Average Pooling)**:
    特徴マップ全体の平均を計算して出力する層。これにより、シーケンスや画像の情報を圧縮し、過剰適合を防ぐ手助けがされる。

これらの用語は、特に実務経験がない初学者にとっては馴染みが薄いものが多いですが、理解が深まると同時に機械学習や深層学習の実践的なスキル向上に役立ちます。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

<center><img src="https://keras.io/img/logo-small.png" alt="Keras logo" width="100"><br/>
This starter notebook is provided by the Keras team.</center>

# LMSYS - Chatbot Arena Human Preference Predictions with [KerasNLP](https://github.com/keras-team/keras-nlp) and [Keras](https://github.com/keras-team/keras)

<div align="center">
    <img src="https://i.ibb.co/wJMF5HL/lmsys.png">
</div>

In this competition, our aim is to predict which LLM responses users will prefer in a head-to-head battle between chatbots powered by large language models (LLMs). In other words, the goal of the competition is to predict the preferences of the judges and determine the likelihood that a given prompt/response pair is selected as the winner. This notebook will guide you through the process of fine-tuning the **DebertaV3** model for this competition using the **Shared Weight** strategy with KerasNLP. This strategy is similar to how Multiple Choice Question (MCQ) models are trained. Additionally, we will use mixed precision for faster training and inference.

**Did you know**: This notebook is backend-agnostic, which means it supports TensorFlow, PyTorch, and JAX backends. However, the best performance can be achieved with `JAX`. KerasNLP and Keras enable the choice of the preferred backend. Explore further details on [Keras](https://keras.io/keras_3/).

**Note**: For a deeper understanding of KerasNLP, refer to the [KerasNLP guides](https://keras.io/keras_nlp/).


# 📚 | Import Libraries 

</div>
<div class="column-right">

# 日本語訳

<center><img src="https://keras.io/img/logo-small.png" alt="Kerasのロゴ" width="100"><br/>
このスターターノートブックはKerasチームによって提供されています。</center>

# LMSYS - Chatbot Arena 人間による好み予測 [KerasNLP](https://github.com/keras-team/keras-nlp) と [Keras](https://github.com/keras-team/keras) を使用

<div align="center">
    <img src="https://i.ibb.co/wJMF5HL/lmsys.png">
</div>

このコンペティションでは、我々の目的は、LLM（大規模言語モデル）によって動かされるチャットボット同士の対戦において、どのLLMの応答がユーザーに好まれるかを予測することです。言い換えれば、このコンペティションの目標は、審査員の好みを予測し、特定のプロンプト/応答ペアが勝者として選ばれる可能性を決定することです。このノートブックでは、KerasNLPを使用して、このコンペティションのために**DebertaV3**モデルをファインチューニングするプロセスを案内します。この戦略は、選択肢問題（MCQ）モデルがトレーニングされる方法に似ています。さらに、トレーニングと推論を迅速化するために混合精度を使用します。

**ご存知でしたか**: このノートブックはバックエンドに依存しないため、TensorFlow、PyTorch、JAXのいずれのバックエンドにも対応しています。ただし、最良のパフォーマンスを達成するには `JAX` を使用することをお勧めします。KerasNLPとKerasは、好みのバックエンドを選択することを可能にしています。さらなる詳細は[Keras](https://keras.io/keras_3/)で確認してください。

**注意**: KerasNLPについてのより深い理解を得るためには、[KerasNLPガイド](https://keras.io/keras_nlp/)を参照してください。

# 📚 | ライブラリのインポート

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import os
os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"

import keras_nlp
import keras
import tensorflow as tf

import numpy as np 
import pandas as pd
from tqdm import tqdm
import json

import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px
```

</div>
<div class="column-right">

# 日本語訳

```python
import os
# Kerasのバックエンドを"jax"に設定します。
# "tensorflow" または "torch" を使用することもできます。
os.environ["KERAS_BACKEND"] = "jax"  # または "tensorflow" または "torch"

# KerasNLPライブラリをインポートします。
import keras_nlp
# Kerasライブラリをインポートします。
import keras
# TensorFlowライブラリをインポートします。
import tensorflow as tf

# 数値計算のためのNumPyライブラリをインポートします。
import numpy as np 
# データ操作のためのPandasライブラリをインポートします。
import pandas as pd
# プログレスバーの表示のためのtqdmライブラリをインポートします。
from tqdm import tqdm
# JSON操作のためのjsonライブラリをインポートします。
import json

# グラフ描画のためのMatplotlibライブラリをインポートします。
import matplotlib.pyplot as plt
# Matplotlibの設定を行うためのmplモジュールをインポートします。
import matplotlib as mpl
# Plotlyでインタラクティブなグラフを作成するためのexpressモジュールをインポートします。
import plotly.express as px
```

</div>
</details>

In [ ]:
import os
# Kerasのバックエンドを"jax"に設定します。
# "tensorflow" または "torch" を使用することもできます。
os.environ["KERAS_BACKEND"] = "jax"  # または "tensorflow" または "torch"

# KerasNLPライブラリをインポートします。
import keras_nlp
# Kerasライブラリをインポートします。
import keras
# TensorFlowライブラリをインポートします。
import tensorflow as tf

# 数値計算のためのNumPyライブラリをインポートします。
import numpy as np 
# データ操作のためのPandasライブラリをインポートします。
import pandas as pd
# プログレスバーの表示のためのtqdmライブラリをインポートします。
from tqdm import tqdm
# JSON操作のためのjsonライブラリをインポートします。
import json

# グラフ描画のためのMatplotlibライブラリをインポートします。
import matplotlib.pyplot as plt
# Matplotlibの設定を行うためのmplモジュールをインポートします。
import matplotlib as mpl
# Plotlyでインタラクティブなグラフを作成するためのexpressモジュールをインポートします。
import plotly.express as px

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Library Version

</div>
<div class="column-right">

# 日本語訳

## ライブラリのバージョン

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)
print("KerasNLP:", keras_nlp.__version__)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 現在のTensorFlowのバージョンを出力します。
print("TensorFlow:", tf.__version__)
# 現在のKerasのバージョンを出力します。
print("Keras:", keras.__version__)
# 現在のKerasNLPのバージョンを出力します。
print("KerasNLP:", keras_nlp.__version__)
```

</div>
</details>

In [ ]:
# 現在のTensorFlowのバージョンを出力します。
print("TensorFlow:", tf.__version__)
# 現在のKerasのバージョンを出力します。
print("Keras:", keras.__version__)
# 現在のKerasNLPのバージョンを出力します。
print("KerasNLP:", keras_nlp.__version__)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# ⚙️ | Configuration

</div>
<div class="column-right">

# 日本語訳

# ⚙️ | 設定

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
class CFG:
    seed = 42  # Random seed
    preset = "deberta_v3_extra_small_en" # Name of pretrained models
    sequence_length = 512  # Input sequence length
    epochs = 3 # Training epochs
    batch_size = 16  # Batch size
    scheduler = 'cosine'  # Learning rate scheduler
    label2name = {0: 'winner_model_a', 1: 'winner_model_b', 2: 'winner_tie'}
    name2label = {v:k for k, v in label2name.items()}
    class_labels = list(label2name.keys())
    class_names = list(label2name.values())
```

</div>
<div class="column-right">

# 日本語訳

```python
class CFG:
    # ランダムシードを設定します。
    seed = 42  # ランダムシード
    # 使用する事前トレーニングモデルの名前を指定します。
    preset = "deberta_v3_extra_small_en" # 事前トレーニングモデルの名前
    # 入力シーケンスの長さを設定します。
    sequence_length = 512  # 入力シーケンスの長さ
    # トレーニングエポックの数を設定します。
    epochs = 3 # トレーニングエポック数
    # バッチサイズを指定します。
    batch_size = 16  # バッチサイズ
    # 学習率スケジューラのタイプを設定します。
    scheduler = 'cosine'  # 学習率スケジューラ
    # ラベルとその名前の対応を定義します。
    label2name = {0: 'winner_model_a', 1: 'winner_model_b', 2: 'winner_tie'}
    # 名前からラベルを取得する辞書を作成します。
    name2label = {v:k for k, v in label2name.items()}
    # クラスラベルのリストを作成します。
    class_labels = list(label2name.keys())
    # クラス名のリストを作成します。
    class_names = list(label2name.values())
```

</div>
</details>

In [ ]:
class CFG:
    # ランダムシードを設定します。
    seed = 42  # ランダムシード
    # 使用する事前トレーニングモデルの名前を指定します。
    preset = "deberta_v3_extra_small_en" # 事前トレーニングモデルの名前
    # 入力シーケンスの長さを設定します。
    sequence_length = 512  # 入力シーケンスの長さ
    # トレーニングエポックの数を設定します。
    epochs = 3 # トレーニングエポック数
    # バッチサイズを指定します。
    batch_size = 16  # バッチサイズ
    # 学習率スケジューラのタイプを設定します。
    scheduler = 'cosine'  # 学習率スケジューラ
    # ラベルとその名前の対応を定義します。
    label2name = {0: 'winner_model_a', 1: 'winner_model_b', 2: 'winner_tie'}
    # 名前からラベルを取得する辞書を作成します。
    name2label = {v:k for k, v in label2name.items()}
    # クラスラベルのリストを作成します。
    class_labels = list(label2name.keys())
    # クラス名のリストを作成します。
    class_names = list(label2name.values())

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# ♻️ | Reproducibility 
Sets value for random seed to produce similar result in each run.

</div>
<div class="column-right">

# 日本語訳

# ♻️ | 再現性
ランダムシードの値を設定して、各実行で類似した結果を得られるようにします。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
keras.utils.set_random_seed(CFG.seed)
```

</div>
<div class="column-right">

# 日本語訳

```python
# Kerasで使用するランダムシードを設定します。
keras.utils.set_random_seed(CFG.seed)
```

</div>
</details>

In [ ]:
# Kerasで使用するランダムシードを設定します。
keras.utils.set_random_seed(CFG.seed)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# 🧮 | Mixed Precision

In this notebook, we will use mixed precision instead of float32 precision for training and inference to reduce GPU memory usage. This will ultimately allow us to use larger batch sizes, thus reducing our training and inference time.

</div>
<div class="column-right">

# 日本語訳

# 🧮 | 混合精度

このノートブックでは、トレーニングと推論の際にfloat32精度の代わりに混合精度を使用して、GPUメモリの使用量を削減します。これにより、より大きなバッチサイズを使用できるようになり、トレーニングと推論の時間を短縮することができます。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
keras.mixed_precision.set_global_policy("mixed_float16")
```

</div>
<div class="column-right">

# 日本語訳

```python
# Kerasの全体的なポリシーを混合精度（mixed_float16）に設定します。
keras.mixed_precision.set_global_policy("mixed_float16")
```

</div>
</details>

In [ ]:
# Kerasの全体的なポリシーを混合精度（mixed_float16）に設定します。
keras.mixed_precision.set_global_policy("mixed_float16")

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# 📁 | Dataset Path 

</div>
<div class="column-right">

# 日本語訳

# 📁 | データセットのパス

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
BASE_PATH = '/kaggle/input/lmsys-chatbot-arena'
```

</div>
<div class="column-right">

# 日本語訳

```python
# データセットのベースパスを設定します。
BASE_PATH = '/kaggle/input/lmsys-chatbot-arena'
```

</div>
</details>

In [ ]:
# データセットのベースパスを設定します。
BASE_PATH = '/kaggle/input/lmsys-chatbot-arena'

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# 📖 | Meta Data 

The competition dataset comprises user interactions from the ChatBot Arena. In each interaction, a judge presents one or more prompts to two different large language models and then indicates which model provided the more satisfactory response. The training data contains `55,000` rows, with an expected `25,000` rows in the test set.

## Files

### `train.csv`
- `id`: Unique identifier for each row.
- `model_[a/b]`: Model identity, present in train.csv but not in test.csv.
- `prompt`: Input prompt given to both models.
- `response_[a/b]`: Model_[a/b]'s response to the prompt.
- `winner_model_[a/b/tie]`: Binary columns indicating the judge's selection (ground truth target).

### `test.csv`
- `id`: Unique identifier for each row.
- `prompt`: Input prompt given to both models.
- `response_[a/b]`: Model_[a/b]'s response to the prompt.

> Note that each interaction may have multiple prompts and responses, but this notebook will use only **one prompt per interaction**. You can choose to use all prompts and responses. Additionally, prompts and responses in the dataframe are provided as string-formatted lists, so they need to be converted to literal lists using `eval()`.


## Train Data

</div>
<div class="column-right">

# 日本語訳

# 📖 | メタデータ

コンペティションのデータセットは、ChatBot Arenaからのユーザーのインタラクションで構成されています。各インタラクションでは、審査員が2つの異なる大規模言語モデルに対して1つ以上のプロンプトを提示し、どのモデルがより満足のいく応答を提供したかを示します。トレーニングデータには`55,000`行が含まれており、テストセットには約`25,000`行が期待されています。

## ファイル

### `train.csv`
- `id`: 各行の一意の識別子。
- `model_[a/b]`: モデルの識別子。train.csvには存在しますが、test.csvには存在しません。
- `prompt`: 両方のモデルに与えられる入力プロンプト。
- `response_[a/b]`: モデル_[a/b]のプロンプトに対する応答。
- `winner_model_[a/b/tie]`: 審査員の選択（正解ターゲット）を示すバイナリ列。

### `test.csv`
- `id`: 各行の一意の識別子。
- `prompt`: 両方のモデルに与えられる入力プロンプト。
- `response_[a/b]`: モデル_[a/b]のプロンプトに対する応答。

> 注意: 各インタラクションには複数のプロンプトと応答がある場合がありますが、このノートブックでは**各インタラクションにつき1つのプロンプト**のみを使用します。すべてのプロンプトと応答を使用することもできます。また、データフレーム内のプロンプトと応答は文字列形式のリストとして提供されているため、`eval()`を使用してリテラルリストに変換する必要があります。

## トレーニングデータ

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Load Train Data
df = pd.read_csv(f'{BASE_PATH}/train.csv') 

# Sample data
# df = df.sample(frac=0.10)

# Take the first prompt and its associated response
df["prompt"] = df.prompt.map(lambda x: eval(x)[0])
df["response_a"] = df.response_a.map(lambda x: eval(x.replace("null","''"))[0])
df["response_b"] = df.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

# Label conversion
df["class_name"] = df[["winner_model_a", "winner_model_b" , "winner_tie"]].idxmax(axis=1)
df["class_label"] = df.class_name.map(CFG.name2label)

# Show Sample
df.head()
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニングデータを読み込みます。
df = pd.read_csv(f'{BASE_PATH}/train.csv') 

# サンプルデータ
# df = df.sample(frac=0.10)

# 最初のプロンプトとその関連する応答を取得します。
df["prompt"] = df.prompt.map(lambda x: eval(x)[0])
df["response_a"] = df.response_a.map(lambda x: eval(x.replace("null","''"))[0])
df["response_b"] = df.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

# ラベルの変換を行います。
df["class_name"] = df[["winner_model_a", "winner_model_b" , "winner_tie"]].idxmax(axis=1)
df["class_label"] = df.class_name.map(CFG.name2label)

# サンプルを表示します。
df.head()
```

</div>
</details>

In [ ]:
# トレーニングデータを読み込みます。
df = pd.read_csv(f'{BASE_PATH}/train.csv') 

# サンプルデータ
# df = df.sample(frac=0.10)

# 最初のプロンプトとその関連する応答を取得します。
df["prompt"] = df.prompt.map(lambda x: eval(x)[0])
df["response_a"] = df.response_a.map(lambda x: eval(x.replace("null","''"))[0])
df["response_b"] = df.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

# ラベルの変換を行います。
df["class_name"] = df[["winner_model_a", "winner_model_b" , "winner_tie"]].idxmax(axis=1)
df["class_label"] = df.class_name.map(CFG.name2label)

# サンプルを表示します。
df.head()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Test Data

</div>
<div class="column-right">

# 日本語訳

## テストデータ

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Load Test Data
test_df = pd.read_csv(f'{BASE_PATH}/test.csv')

# Take the first prompt and response
test_df["prompt"] = test_df.prompt.map(lambda x: eval(x)[0])
test_df["response_a"] = test_df.response_a.map(lambda x: eval(x.replace("null","''"))[0])
test_df["response_b"] = test_df.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

# Show Sample
test_df.head()
```

</div>
<div class="column-right">

# 日本語訳

```python
# テストデータを読み込みます。
test_df = pd.read_csv(f'{BASE_PATH}/test.csv')

# 最初のプロンプトと応答を取得します。
test_df["prompt"] = test_df.prompt.map(lambda x: eval(x)[0])
test_df["response_a"] = test_df.response_a.map(lambda x: eval(x.replace("null","''"))[0])
test_df["response_b"] = test_df.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

# サンプルを表示します。
test_df.head()
```

</div>
</details>

In [ ]:
# テストデータを読み込みます。
test_df = pd.read_csv(f'{BASE_PATH}/test.csv')

# 最初のプロンプトと応答を取得します。
test_df["prompt"] = test_df.prompt.map(lambda x: eval(x)[0])
test_df["response_a"] = test_df.response_a.map(lambda x: eval(x.replace("null","''"))[0])
test_df["response_b"] = test_df.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

# サンプルを表示します。
test_df.head()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Contextualize Response with Prompt

In our approach, we will contextualize each response with the prompt instead of using a single prompt for all responses. This means that for each response, we will provide the model with the same set of prompts combined with their respective response (e.g., `(P + R_A)`, `(P + R_B)`, etc.). This approach is similar to the multiple-choice question task in NLP.

> Note that some prompts and responses may not be encoded with `utf-8`, resulting in errors when creating the dataloader. In such cases, we will replace them with an empty string.


</div>
<div class="column-right">

# 日本語訳

## プロンプトによる応答の文脈化

我々のアプローチでは、すべての応答に対して単一のプロンプトを使用するのではなく、各応答をプロンプトで文脈化します。つまり、各応答に対して、モデルにそれぞれの応答と組み合わせた同じセットのプロンプトを提供することになります（例: `(P + R_A)`、`(P + R_B)`など）。このアプローチは、NLPにおける選択肢問題のタスクに類似しています。

> 注意: 一部のプロンプトや応答が`utf-8`でエンコードされていない場合、データローダーを作成する際にエラーが発生することがあります。その場合、空の文字列に置き換えます。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Define a function to create options based on the prompt and choices
def make_pairs(row):
    row["encode_fail"] = False
    try:
        prompt = row.prompt.encode("utf-8").decode("utf-8")
    except:
        prompt = ""
        row["encode_fail"] = True

    try:
        response_a = row.response_a.encode("utf-8").decode("utf-8")
    except:
        response_a = ""
        row["encode_fail"] = True

    try:
        response_b = row.response_b.encode("utf-8").decode("utf-8")
    except:
        response_b = ""
        row["encode_fail"] = True
        
    row['options'] = [f"Prompt: {prompt}\n\nResponse: {response_a}",  # Response from Model A
                      f"Prompt: {prompt}\n\nResponse: {response_b}"  # Response from Model B
                     ]
    return row

```

</div>
<div class="column-right">

# 日本語訳

```python
# プロンプトと選択肢に基づいてオプションを作成する関数を定義します。
def make_pairs(row):
    row["encode_fail"] = False
    try:
        # プロンプトをUTF-8でエンコードし、デコードします。
        prompt = row.prompt.encode("utf-8").decode("utf-8")
    except:
        # エンコードに失敗した場合は空の文字列にします。
        prompt = ""
        row["encode_fail"] = True

    try:
        # モデルAの応答をUTF-8でエンコードし、デコードします。
        response_a = row.response_a.encode("utf-8").decode("utf-8")
    except:
        # エンコードに失敗した場合は空の文字列にします。
        response_a = ""
        row["encode_fail"] = True

    try:
        # モデルBの応答をUTF-8でエンコードし、デコードします。
        response_b = row.response_b.encode("utf-8").decode("utf-8")
    except:
        # エンコードに失敗した場合は空の文字列にします。
        response_b = ""
        row["encode_fail"] = True
        
    # プロンプトと応答を組み合わせたオプションを作成します。
    row['options'] = [f"Prompt: {prompt}\n\nResponse: {response_a}",  # モデルAからの応答
                      f"Prompt: {prompt}\n\nResponse: {response_b}"  # モデルBからの応答
                     ]
    return row
```

</div>
</details>

In [ ]:
# プロンプトと選択肢に基づいてオプションを作成する関数を定義します。
def make_pairs(row):
    row["encode_fail"] = False
    try:
        # プロンプトをUTF-8でエンコードし、デコードします。
        prompt = row.prompt.encode("utf-8").decode("utf-8")
    except:
        # エンコードに失敗した場合は空の文字列にします。
        prompt = ""
        row["encode_fail"] = True

    try:
        # モデルAの応答をUTF-8でエンコードし、デコードします。
        response_a = row.response_a.encode("utf-8").decode("utf-8")
    except:
        # エンコードに失敗した場合は空の文字列にします。
        response_a = ""
        row["encode_fail"] = True

    try:
        # モデルBの応答をUTF-8でエンコードし、デコードします。
        response_b = row.response_b.encode("utf-8").decode("utf-8")
    except:
        # エンコードに失敗した場合は空の文字列にします。
        response_b = ""
        row["encode_fail"] = True
        
    # プロンプトと応答を組み合わせたオプションを作成します。
    row['options'] = [f"Prompt: {prompt}\n\nResponse: {response_a}",  # モデルAからの応答
                      f"Prompt: {prompt}\n\nResponse: {response_b}"  # モデルBからの応答
                     ]
    return row

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
df = df.apply(make_pairs, axis=1)  # Apply the make_pairs function to each row in df
display(df.head(2))  # Display the first 2 rows of df

test_df = test_df.apply(make_pairs, axis=1)  # Apply the make_pairs function to each row in df
display(test_df.head(2))  # Display the first 2 rows of df
```

</div>
<div class="column-right">

# 日本語訳

```python
# make_pairs関数をdfの各行に適用します。
df = df.apply(make_pairs, axis=1)  
# dfの最初の2行を表示します。
display(df.head(2))  

# make_pairs関数をtest_dfの各行に適用します。
test_df = test_df.apply(make_pairs, axis=1)  
# test_dfの最初の2行を表示します。
display(test_df.head(2))
```

</div>
</details>

In [ ]:
# make_pairs関数をdfの各行に適用します。
df = df.apply(make_pairs, axis=1)  
# dfの最初の2行を表示します。
display(df.head(2))  

# make_pairs関数をtest_dfの各行に適用します。
test_df = test_df.apply(make_pairs, axis=1)  
# test_dfの最初の2行を表示します。
display(test_df.head(2))

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Encoding Fail Statistics

Let's examine how many samples have encoding issues. From the code below, we can see that only $1\%$ of the samples failed to be encoded, while $99\%$ of the samples don't have any issues. A similar pattern can be expected for the test data as well. Thus, considering empty strings for this small portion of the data will not have much impact on our training and inference.

</div>
<div class="column-right">

# 日本語訳

## エンコーディング失敗統計

エンコーディングの問題があるサンプル数を確認してみましょう。以下のコードから、エンコーディングに失敗したサンプルはわずか$1\%$であり、$99\%$のサンプルには問題がないことがわかります。テストデータでも同様の傾向が期待できます。このため、データのごく一部を空の文字列として扱っても、トレーニングや推論に大きな影響はないでしょう。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
df.encode_fail.value_counts(normalize=False)
```

</div>
<div class="column-right">

# 日本語訳

```python
# エンコーディング失敗の統計をカウントします。
df.encode_fail.value_counts(normalize=False)
```

</div>
</details>

In [ ]:
# エンコーディング失敗の統計をカウントします。
df.encode_fail.value_counts(normalize=False)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# 🎨 | Exploratory Data Analysis (EDA)

## LLM Distribution

</div>
<div class="column-right">

# 日本語訳

# 🎨 | 探索的データ分析 (EDA)

## LLMの分布

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model_df = pd.concat([df.model_a, df.model_b])
counts = model_df.value_counts().reset_index()
counts.columns = ['LLM', 'Count']

# Create a bar plot with custom styling using Plotly
fig = px.bar(counts, x='LLM', y='Count',
             title='Distribution of LLMs',
             color='Count', color_continuous_scale='viridis')

fig.update_layout(xaxis_tickangle=-45)  # Rotate x-axis labels for better readability

fig.show()

```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルAとモデルBのデータを結合します。
model_df = pd.concat([df.model_a, df.model_b])
# 各モデルのカウントを取得します。
counts = model_df.value_counts().reset_index()
counts.columns = ['LLM', 'Count']

# Plotlyを使用してカスタムスタイリングの棒グラフを作成します。
fig = px.bar(counts, x='LLM', y='Count',
             title='LLMの分布',
             color='Count', color_continuous_scale='viridis')

# x軸ラベルを回転させて、読みやすくします。
fig.update_layout(xaxis_tickangle=-45)  

# グラフを表示します。
fig.show()
```

</div>
</details>

In [ ]:
# モデルAとモデルBのデータを結合します。
model_df = pd.concat([df.model_a, df.model_b])
# 各モデルのカウントを取得します。
counts = model_df.value_counts().reset_index()
counts.columns = ['LLM', 'Count']

# Plotlyを使用してカスタムスタイリングの棒グラフを作成します。
fig = px.bar(counts, x='LLM', y='Count',
             title='LLMの分布',
             color='Count', color_continuous_scale='viridis')

# x軸ラベルを回転させて、読みやすくします。
fig.update_layout(xaxis_tickangle=-45)  

# グラフを表示します。
fig.show()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Winning Distribution

</div>
<div class="column-right">

# 日本語訳

## 勝利の分布

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
counts = df['class_name'].value_counts().reset_index()
counts.columns = ['Winner', 'Win Count']

fig = px.bar(counts, x='Winner', y='Win Count',
             title='Winner distribution for Train Data',
             labels={'Winner': 'Winner', 'Win Count': 'Win Count'},
             color='Winner', color_continuous_scale='viridis')

fig.update_layout(xaxis_title="Winner", yaxis_title="Win Count")

fig.show()

```

</div>
<div class="column-right">

# 日本語訳

```python
# 勝者のカウントを取得します。
counts = df['class_name'].value_counts().reset_index()
counts.columns = ['Winner', 'Win Count']

# トレーニングデータの勝者分布を示す棒グラフを作成します。
fig = px.bar(counts, x='Winner', y='Win Count',
             title='トレーニングデータの勝者分布',
             labels={'Winner': '勝者', 'Win Count': '勝利の数'},
             color='Winner', color_continuous_scale='viridis')

# x軸とy軸のタイトルを設定します。
fig.update_layout(xaxis_title="勝者", yaxis_title="勝利の数")

# グラフを表示します。
fig.show()
```

</div>
</details>

In [ ]:
# 勝者のカウントを取得します。
counts = df['class_name'].value_counts().reset_index()
counts.columns = ['Winner', 'Win Count']

# トレーニングデータの勝者分布を示す棒グラフを作成します。
fig = px.bar(counts, x='Winner', y='Win Count',
             title='トレーニングデータの勝者分布',
             labels={'Winner': '勝者', 'Win Count': '勝利の数'},
             color='Winner', color_continuous_scale='viridis')

# x軸とy軸のタイトルを設定します。
fig.update_layout(xaxis_title="勝者", yaxis_title="勝利の数")

# グラフを表示します。
fig.show()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# 🔪 | Data Split

In the code snippet provided below, we will divide the existing data into training and validation using a stratification of `class_label` column.

</div>
<div class="column-right">

# 日本語訳

# 🔪 | データ分割

以下のコードスニペットでは、`class_label`列の層別化を使用して、既存のデータをトレーニングデータと検証データに分割します。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from sklearn.model_selection import train_test_split  # Import package

train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df["class_label"])
```

</div>
<div class="column-right">

# 日本語訳

```python
# パッケージをインポートします。
from sklearn.model_selection import train_test_split  # パッケージをインポート

# データをトレーニングデータと検証データに分割します。
train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df["class_label"])
```

</div>
</details>

In [ ]:
# パッケージをインポートします。
from sklearn.model_selection import train_test_split  # パッケージをインポート

# データをトレーニングデータと検証データに分割します。
train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df["class_label"])

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# 🍽️ | Preprocessing

**What it does:** The preprocessor takes input strings and transforms them into a dictionary (`token_ids`, `padding_mask`) containing preprocessed tensors. This process starts with tokenization, where input strings are converted into sequences of token IDs.

**Why it's important:** Initially, raw text data is complex and challenging for modeling due to its high dimensionality. By converting text into a compact set of tokens, such as transforming `"The quick brown fox"` into `["the", "qu", "##ick", "br", "##own", "fox"]`, we simplify the data. Many models rely on special tokens and additional tensors to understand input. These tokens help divide input and identify padding, among other tasks. Making all sequences the same length through padding boosts computational efficiency, making subsequent steps smoother.

Explore the following pages to access the available preprocessing and tokenizer layers in **KerasNLP**:
- [Preprocessing](https://keras.io/api/keras_nlp/preprocessing_layers/)
- [Tokenizers](https://keras.io/api/keras_nlp/tokenizers/)

</div>
<div class="column-right">

# 日本語訳

# 🍽️ | 前処理

**何をするのか:** 前処理器は入力文字列を受け取り、それを前処理されたテンソルを含む辞書（`token_ids`, `padding_mask`）に変換します。このプロセスはトークン化から始まり、入力文字列がトークンIDのシーケンスに変換されます。

**なぜ重要なのか:** 生のテキストデータは、その高次元性のためにモデル化が難しく、複雑です。テキストをトークンのコンパクトなセットに変換することで、例えば `"The quick brown fox"` を `["the", "qu", "##ick", "br", "##own", "fox"]` に変換することで、データを単純化します。多くのモデルは、特別なトークンや追加のテンソルを使用して入力を理解します。これらのトークンは、入力を分割し、パディングを特定するなどのタスクに役立ちます。すべてのシーケンスをパディングを使って同じ長さにすることで、計算効率が向上し、その後のステップがスムーズになります。

**KerasNLP**で利用可能な前処理およびトークナイザー層にアクセスするには、以下のページを確認してください:
- [前処理](https://keras.io/api/keras_nlp/preprocessing_layers/)
- [トークナイザー](https://keras.io/api/keras_nlp/tokenizers/)

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
preprocessor = keras_nlp.models.DebertaV3Preprocessor.from_preset(
    preset=CFG.preset, # Name of the model
    sequence_length=CFG.sequence_length, # Max sequence length, will be padded if shorter
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# DebertaV3モデルの前処理器を設定します。
preprocessor = keras_nlp.models.DebertaV3Preprocessor.from_preset(
    preset=CFG.preset, # モデルの名前
    sequence_length=CFG.sequence_length, # 最大シーケンス長、短い場合はパディングされる
)
```

</div>
</details>

In [ ]:
# DebertaV3モデルの前処理器を設定します。
preprocessor = keras_nlp.models.DebertaV3Preprocessor.from_preset(
    preset=CFG.preset, # モデルの名前
    sequence_length=CFG.sequence_length, # 最大シーケンス長、短い場合はパディングされる
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Now, let's examine what the output shape of the preprocessing layer looks like. The output shape of the layer can be represented as $(num\_responses, sequence\_length)$.

</div>
<div class="column-right">

# 日本語訳

次に、前処理層の出力形状がどのようになるかを確認しましょう。層の出力形状は $(num\_responses, sequence\_length)$ として表されます。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
outs = preprocessor(df.options.iloc[0])  # Process options for the first row

# Display the shape of each processed output
for k, v in outs.items():
    print(k, ":", v.shape)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 最初の行のオプションを処理します。
outs = preprocessor(df.options.iloc[0])  

# 各処理された出力の形状を表示します。
for k, v in outs.items():
    print(k, ":", v.shape)
```

</div>
</details>

In [ ]:
# 最初の行のオプションを処理します。
outs = preprocessor(df.options.iloc[0])  

# 各処理された出力の形状を表示します。
for k, v in outs.items():
    print(k, ":", v.shape)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

We'll use the `preprocessing_fn` function to transform each text option using the `dataset.map(preprocessing_fn)` method.

</div>
<div class="column-right">

# 日本語訳

`preprocessing_fn`関数を使用して、`dataset.map(preprocessing_fn)`メソッドを介して各テキストオプションを変換します。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def preprocess_fn(text, label=None):
    text = preprocessor(text)  # Preprocess text
    return (text, label) if label is not None else text  # Return processed text and label if available
```

</div>
<div class="column-right">

# 日本語訳

```python
# テキストを前処理するための関数を定義します。
def preprocess_fn(text, label=None):
    # テキストを前処理します。
    text = preprocessor(text)  
    # ラベルが利用可能であれば、処理されたテキストとラベルを返します。
    return (text, label) if label is not None else text
```

</div>
</details>

In [ ]:
# テキストを前処理するための関数を定義します。
def preprocess_fn(text, label=None):
    # テキストを前処理します。
    text = preprocessor(text)  
    # ラベルが利用可能であれば、処理されたテキストとラベルを返します。
    return (text, label) if label is not None else text

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# 🍚 | DataLoader

The code below sets up a robust data flow pipeline using `tf.data.Dataset` for data processing. Notable aspects of `tf.data` include its ability to simplify pipeline construction and represent components in sequences.

To learn more about `tf.data`, refer to this [documentation](https://www.tensorflow.org/guide/data).

</div>
<div class="column-right">

# 日本語訳

# 🍚 | データローダー

以下のコードは、データ処理のために`tf.data.Dataset`を使用して堅牢なデータフローパイプラインを構築します。`tf.data`の主な特徴は、パイプラインの構築を簡素化し、コンポーネントをシーケンスで表現できる点です。

`tf.data`に関する詳細は、この[ドキュメント](https://www.tensorflow.org/guide/data)を参照してください。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def build_dataset(texts, labels=None, batch_size=32,
                  cache=True, shuffle=1024):
    AUTO = tf.data.AUTOTUNE  # AUTOTUNE option
    slices = (texts,) if labels is None else (texts, keras.utils.to_categorical(labels, num_classes=3))  # Create slices
    ds = tf.data.Dataset.from_tensor_slices(slices)  # Create dataset from slices
    ds = ds.cache() if cache else ds  # Cache dataset if enabled
    ds = ds.map(preprocess_fn, num_parallel_calls=AUTO)  # Map preprocessing function
    opt = tf.data.Options()  # Create dataset options
    if shuffle: 
        ds = ds.shuffle(shuffle, seed=CFG.seed)  # Shuffle dataset if enabled
        opt.experimental_deterministic = False
    ds = ds.with_options(opt)  # Set dataset options
    ds = ds.batch(batch_size, drop_remainder=False)  # Batch dataset
    ds = ds.prefetch(AUTO)  # Prefetch next batch
    return ds  # Return the built dataset
```

</div>
<div class="column-right">

# 日本語訳

```python
# テキストとラベルを使用してデータセットを構築する関数を定義します。
def build_dataset(texts, labels=None, batch_size=32,
                  cache=True, shuffle=1024):
    AUTO = tf.data.AUTOTUNE  # AUTOTUNEオプション
    # ラベルが指定されていない場合、slicesをテキストのタプルに設定します。
    slices = (texts,) if labels is None else (texts, keras.utils.to_categorical(labels, num_classes=3))  # スライスを作成
    # スライスからデータセットを作成します。
    ds = tf.data.Dataset.from_tensor_slices(slices)  
    # キャッシュが有効な場合、データセットをキャッシュします。
    ds = ds.cache() if cache else ds  
    # 前処理関数をマッピングします。
    ds = ds.map(preprocess_fn, num_parallel_calls=AUTO)  
    opt = tf.data.Options()  # データセットオプションを作成
    if shuffle: 
        # シャッフルが有効な場合、データセットをシャッフルします。
        ds = ds.shuffle(shuffle, seed=CFG.seed)  
        opt.experimental_deterministic = False
    # データセットオプションを設定します。
    ds = ds.with_options(opt)  
    # データセットをバッチ化します。
    ds = ds.batch(batch_size, drop_remainder=False)  
    # 次のバッチをプリフェッチします。
    ds = ds.prefetch(AUTO)  
    return ds  # 構築されたデータセットを返します
```

</div>
</details>

In [ ]:
# テキストとラベルを使用してデータセットを構築する関数を定義します。
def build_dataset(texts, labels=None, batch_size=32,
                  cache=True, shuffle=1024):
    AUTO = tf.data.AUTOTUNE  # AUTOTUNEオプション
    # ラベルが指定されていない場合、slicesをテキストのタプルに設定します。
    slices = (texts,) if labels is None else (texts, keras.utils.to_categorical(labels, num_classes=3))  # スライスを作成
    # スライスからデータセットを作成します。
    ds = tf.data.Dataset.from_tensor_slices(slices)  
    # キャッシュが有効な場合、データセットをキャッシュします。
    ds = ds.cache() if cache else ds  
    # 前処理関数をマッピングします。
    ds = ds.map(preprocess_fn, num_parallel_calls=AUTO)  
    opt = tf.data.Options()  # データセットオプションを作成
    if shuffle: 
        # シャッフルが有効な場合、データセットをシャッフルします。
        ds = ds.shuffle(shuffle, seed=CFG.seed)  
        opt.experimental_deterministic = False
    # データセットオプションを設定します。
    ds = ds.with_options(opt)  
    # データセットをバッチ化します。
    ds = ds.batch(batch_size, drop_remainder=False)  
    # 次のバッチをプリフェッチします。
    ds = ds.prefetch(AUTO)  
    return ds  # 構築されたデータセットを返します

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Build Train/Valid Dataloader

</div>
<div class="column-right">

# 日本語訳

## トレーニング/検証データローダーの構築

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Train
train_texts = train_df.options.tolist()  # Extract training texts
train_labels = train_df.class_label.tolist()  # Extract training labels
train_ds = build_dataset(train_texts, train_labels,
                         batch_size=CFG.batch_size,
                         shuffle=True)

# Valid
valid_texts = valid_df.options.tolist()  # Extract validation texts
valid_labels = valid_df.class_label.tolist()  # Extract validation labels
valid_ds = build_dataset(valid_texts, valid_labels,
                         batch_size=CFG.batch_size,
                         shuffle=False)
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニングデータのテキストとラベルを抽出します。
train_texts = train_df.options.tolist()  # トレーニングテキストを抽出
train_labels = train_df.class_label.tolist()  # トレーニングラベルを抽出
# トレーニングデータローダーを構築します。
train_ds = build_dataset(train_texts, train_labels,
                         batch_size=CFG.batch_size,
                         shuffle=True)

# 検証データのテキストとラベルを抽出します。
valid_texts = valid_df.options.tolist()  # 検証テキストを抽出
valid_labels = valid_df.class_label.tolist()  # 検証ラベルを抽出
# 検証データローダーを構築します。
valid_ds = build_dataset(valid_texts, valid_labels,
                         batch_size=CFG.batch_size,
                         shuffle=False)
```

</div>
</details>

In [ ]:
# トレーニングデータのテキストとラベルを抽出します。
train_texts = train_df.options.tolist()  # トレーニングテキストを抽出
train_labels = train_df.class_label.tolist()  # トレーニングラベルを抽出
# トレーニングデータローダーを構築します。
train_ds = build_dataset(train_texts, train_labels,
                         batch_size=CFG.batch_size,
                         shuffle=True)

# 検証データのテキストとラベルを抽出します。
valid_texts = valid_df.options.tolist()  # 検証テキストを抽出
valid_labels = valid_df.class_label.tolist()  # 検証ラベルを抽出
# 検証データローダーを構築します。
valid_ds = build_dataset(valid_texts, valid_labels,
                         batch_size=CFG.batch_size,
                         shuffle=False)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# ⚓ | LR Schedule

Implementing a learning rate scheduler is crucial for transfer learning. The learning rate initiates at `lr_start` and gradually tapers down to `lr_min` using various techniques, including:
- `step`: Lowering the learning rate in step-wise manner resembling stairs.
- `cos`: Utilizing a cosine curve to gradually reduce the learning rate.
- `exp`: Exponentially decreasing the learning rate.

**Importance:** A well-structured learning rate schedule is essential for efficient model training, ensuring optimal convergence and avoiding issues such as overshooting or stagnation.

</div>
<div class="column-right">

# 日本語訳

# ⚓ | 学習率スケジュール

学習率スケジューラーの実装は、転移学習において非常に重要です。学習率は`lr_start`から始まり、さまざまな手法を用いて徐々に`lr_min`まで減少します。手法には以下が含まれます:
- `step`: ステップ状に学習率を下げる方法で、階段のように見えます。
- `cos`: コサインカーブを利用して、学習率を徐々に減少させる方法です。
- `exp`: 学習率を指数的に減少させる方法です。

**重要性:** 適切に構築された学習率スケジュールは、効率的なモデルのトレーニングに不可欠であり、最適な収束を保証し、オーバーシュートや停滞といった問題を避けるのに役立ちます。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import math

def get_lr_callback(batch_size=8, mode='cos', epochs=10, plot=False):
    lr_start, lr_max, lr_min = 1.0e-6, 0.6e-6 * batch_size, 1e-6
    lr_ramp_ep, lr_sus_ep, lr_decay = 2, 0, 0.8

    def lrfn(epoch):  # Learning rate update function
        if epoch < lr_ramp_ep: lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep: lr = lr_max
        elif mode == 'exp': lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
        elif mode == 'step': lr = lr_max * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep) // 2)
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep + 3, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        return lr

    if plot:  # Plot lr curve if plot is True
        plt.figure(figsize=(10, 5))
        plt.plot(np.arange(epochs), [lrfn(epoch) for epoch in np.arange(epochs)], marker='o')
        plt.xlabel('epoch'); plt.ylabel('lr')
        plt.title('LR Scheduler')
        plt.show()

    return keras.callbacks.LearningRateScheduler(lrfn, verbose=False)  # Create lr callback
```

</div>
<div class="column-right">

# 日本語訳

```python
import math

# 学習率コールバックを取得する関数を定義します。
def get_lr_callback(batch_size=8, mode='cos', epochs=10, plot=False):
    lr_start, lr_max, lr_min = 1.0e-6, 0.6e-6 * batch_size, 1e-6  # 学習率の開始、最大、最小値を設定
    lr_ramp_ep, lr_sus_ep, lr_decay = 2, 0, 0.8  # 学習率ランプのエポック数、持続エポック数、減衰率を設定

    def lrfn(epoch):  # 学習率更新関数
        if epoch < lr_ramp_ep: 
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start  # ランプアップ
        elif epoch < lr_ramp_ep + lr_sus_ep: 
            lr = lr_max  # 最大値で維持
        elif mode == 'exp': 
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min  # 指数減衰
        elif mode == 'step': 
            lr = lr_max * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep) // 2)  # ステップ減衰
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep + 3, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs  # コサイン減衰
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        return lr

    if plot:  # plotがTrueの場合、学習率曲線をプロットします。
        plt.figure(figsize=(10, 5))
        plt.plot(np.arange(epochs), [lrfn(epoch) for epoch in np.arange(epochs)], marker='o')
        plt.xlabel('epoch'); plt.ylabel('lr')
        plt.title('学習率スケジューラ')
        plt.show()

    return keras.callbacks.LearningRateScheduler(lrfn, verbose=False)  # 学習率コールバックを作成する
```

</div>
</details>

In [ ]:
import math

# 学習率コールバックを取得する関数を定義します。
def get_lr_callback(batch_size=8, mode='cos', epochs=10, plot=False):
    lr_start, lr_max, lr_min = 1.0e-6, 0.6e-6 * batch_size, 1e-6  # 学習率の開始、最大、最小値を設定
    lr_ramp_ep, lr_sus_ep, lr_decay = 2, 0, 0.8  # 学習率ランプのエポック数、持続エポック数、減衰率を設定

    def lrfn(epoch):  # 学習率更新関数
        if epoch < lr_ramp_ep: 
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start  # ランプアップ
        elif epoch < lr_ramp_ep + lr_sus_ep: 
            lr = lr_max  # 最大値で維持
        elif mode == 'exp': 
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min  # 指数減衰
        elif mode == 'step': 
            lr = lr_max * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep) // 2)  # ステップ減衰
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep + 3, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs  # コサイン減衰
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        return lr

    if plot:  # plotがTrueの場合、学習率曲線をプロットします。
        plt.figure(figsize=(10, 5))
        plt.plot(np.arange(epochs), [lrfn(epoch) for epoch in np.arange(epochs)], marker='o')
        plt.xlabel('epoch'); plt.ylabel('lr')
        plt.title('学習率スケジューラ')
        plt.show()

    return keras.callbacks.LearningRateScheduler(lrfn, verbose=False)  # 学習率コールバックを作成する

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
lr_cb = get_lr_callback(CFG.batch_size, plot=True)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 学習率コールバックを取得します。プロットを表示します。
lr_cb = get_lr_callback(CFG.batch_size, plot=True)
```

</div>
</details>

In [ ]:
# 学習率コールバックを取得します。プロットを表示します。
lr_cb = get_lr_callback(CFG.batch_size, plot=True)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# 💾 | Model Checkpointing

The following code will create a callback that will save the best checkpoint of the model during training, which we will use for inference in the submission.

</div>
<div class="column-right">

# 日本語訳

# 💾 | モデルチェックポイント

以下のコードは、トレーニング中にモデルの最良のチェックポイントを保存するコールバックを作成します。このチェックポイントは、提出時の推論に使用します。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
ckpt_cb = keras.callbacks.ModelCheckpoint(f'best_model.weights.h5',
                                          monitor='val_log_loss',
                                          save_best_only=True,
                                          save_weights_only=True,
                                          mode='min')  # Get Model checkpoint callback
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルチェックポイントコールバックを取得します。
ckpt_cb = keras.callbacks.ModelCheckpoint(f'best_model.weights.h5',
                                          monitor='val_log_loss',  # 検証ロスを監視
                                          save_best_only=True,  # 最良のみを保存
                                          save_weights_only=True,  # 重みのみを保存
                                          mode='min')  # 最小値で監視
```

</div>
</details>

In [ ]:
# モデルチェックポイントコールバックを取得します。
ckpt_cb = keras.callbacks.ModelCheckpoint(f'best_model.weights.h5',
                                          monitor='val_log_loss',  # 検証ロスを監視
                                          save_best_only=True,  # 最良のみを保存
                                          save_weights_only=True,  # 重みのみを保存
                                          mode='min')  # 最小値で監視

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# 📏 | Metric

The metric for this competition is **Log Loss**. This metric can be expressed mathematically as,

$$
\text{Log Loss} = -\frac{1}{N} \sum_{i=1}^{N} \left( y_i \log(p_i) + (1 - y_i) \log(1 - p_i) \right)
$$

where $ N $ is the number of samples, $ y_i $ is the true label, and $ p_i $ is the predicted probability of the sample belonging to the positive class.

Note that this metric is similar to categorical cross entropy widely used in classification tasks. Thus, we don't need to implement the loss from scratch. As the Keras library already has an implementation of this metric, we will simply use the metric to monitor performance of our model.


</div>
<div class="column-right">

# 日本語訳

# 📏 | メトリック

このコンペティションのメトリックは**ログロス**です。このメトリックは数学的に次のように表されます。

$$
\text{Log Loss} = -\frac{1}{N} \sum_{i=1}^{N} \left( y_i \log(p_i) + (1 - y_i) \log(1 - p_i) \right)
$$

ここで、$ N $はサンプルの数、$ y_i $は真のラベル、$ p_i $はサンプルが正のクラスに属する予測確率です。

このメトリックは、分類タスクで広く使用されるカテゴリカルクロスエントロピーに似ています。したがって、ロスをゼロから実装する必要はありません。Kerasライブラリにはこのメトリックの実装が既にあるため、単にこのメトリックを使用してモデルの性能を監視します。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
log_loss = keras.metrics.CategoricalCrossentropy(name="log_loss")
```

</div>
<div class="column-right">

# 日本語訳

```python
# ログロスメトリックをKerasのCategoricalCrossentropyとして定義します。
log_loss = keras.metrics.CategoricalCrossentropy(name="log_loss")
```

</div>
</details>

In [ ]:
# ログロスメトリックをKerasのCategoricalCrossentropyとして定義します。
log_loss = keras.metrics.CategoricalCrossentropy(name="log_loss")

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# 🤖 | Modeling

The `KerasNLP` library provides various NLP model architectures such as `Bert`, `Roberta`, `DebertaV3`, and more. While this notebook focuses on `DebertaV3`, you can explore others in the [KerasNLP documentation](https://keras.io/api/keras_nlp/models/). For a deeper understanding, refer to the [getting started guide](https://keras.io/guides/keras_nlp/getting_started/).

Our approach utilizes `keras_nlp.models.DebertaV3Classifier` to process each prompt and response pair, generating output embeddings. We then concatenate these embeddings and pass them through a Pooling layer and a classifier to obtain logits, followed by a `softmax` function for the final output.

When dealing with multiple responses, we use a weight-sharing strategy. This means we provide the model with one response at a time along with the prompt `(P + R_A)`, `(P + R_B)`, etc., using the same model weights for all responses. After obtaining embeddings for all responses, we concatenate them and apply average pooling. Next, we use a `Linear/Dense` layer along with the `Softmax` function as the classifier for the final result. Providing all responses at once would increase text length and complicate model handling. Note that, in the classifier, we use 3 classes for `winner_model_a`, `winner_model_b`, and `draw` cases.

The diagram below illustrates this approach:

<div align="center">
    <img src="https://i.postimg.cc/g0gcvy3f/Kaggle-drawio.png">
</div>

From a coding perspective, note that we use the same model for all responses with shared weights, contrary to the separate models implied in the diagram.

</div>
<div class="column-right">

# 日本語訳

# 🤖 | モデリング

`KerasNLP`ライブラリは、`Bert`、`Roberta`、`DebertaV3`などのさまざまなNLPモデルアーキテクチャを提供します。このノートブックでは`DebertaV3`に焦点を当てていますが、他のモデルは[KerasNLPのドキュメント](https://keras.io/api/keras_nlp/models/)で探索できます。より深い理解のためには、[入門ガイド](https://keras.io/guides/keras_nlp/getting_started/)を参照してください。

我々のアプローチでは、`keras_nlp.models.DebertaV3Classifier`を利用して各プロンプトと応答のペアを処理し、出力埋め込みを生成します。その後、これらの埋め込みを連結し、プーリング層と分類器を通してログitsを取得し、最終的な出力のために`softmax`関数を適用します。

複数の応答を扱う際には、ウェイトシェアリング戦略を使用します。つまり、モデルにはプロンプトとともに1つの応答をずつ提供し、`(P + R_A)`, `(P + R_B)`などの形で全ての応答に対して同じモデルの重みを使用します。全ての応答の埋め込みを取得した後、それらを連結し、平均プーリングを適用します。次に、`Linear/Dense`層と`Softmax`関数を分類器として最終結果を取得します。一度に全ての応答を提供すると、テキストの長さが増加し、モデルの扱いが複雑になります。分類器では`winner_model_a`、`winner_model_b`、および`draw`ケースの3つのクラスを使用することに注意してください。

下の図はこのアプローチを示しています：

<div align="center">
    <img src="https://i.postimg.cc/g0gcvy3f/Kaggle-drawio.png">
</div>

コーディングの観点からは、図に示されているように別々のモデルではなく、共有重みを持つ同一のモデルをすべての応答に使用することに注意してください。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Define input layers
inputs = {
    "token_ids": keras.Input(shape=(2, None), dtype=tf.int32, name="token_ids"),
    "padding_mask": keras.Input(shape=(2, None), dtype=tf.int32, name="padding_mask"),
}
# Create a DebertaV3Classifier backbone
backbone = keras_nlp.models.DebertaV3Backbone.from_preset(
    CFG.preset,
)

# Compute embeddings for first response: (P + R_A) using backbone
response_a = {k: v[:, 0, :] for k, v in inputs.items()}
embed_a = backbone(response_a)

# Compute embeddings for second response: (P + R_B), using the same backbone
response_b = {k: v[:, 1, :] for k, v in inputs.items()}
embed_b = backbone(response_b)

# Compute final output
embeds = keras.layers.Concatenate(axis=-1)([embed_a, embed_b])
embeds = keras.layers.GlobalAveragePooling1D()(embeds)
outputs = keras.layers.Dense(3, activation="softmax", name="classifier")(embeds)
model = keras.Model(inputs, outputs)

# Compile the model with optimizer, loss, and metrics
model.compile(
    optimizer=keras.optimizers.Adam(5e-6),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.02),
    metrics=[
        log_loss,
        keras.metrics.CategoricalAccuracy(name="accuracy"),
    ],
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 入力層を定義します。
inputs = {
    "token_ids": keras.Input(shape=(2, None), dtype=tf.int32, name="token_ids"),
    "padding_mask": keras.Input(shape=(2, None), dtype=tf.int32, name="padding_mask"),
}
# DebertaV3Classifierバックボーンを作成します。
backbone = keras_nlp.models.DebertaV3Backbone.from_preset(
    CFG.preset,
)

# 最初の応答: (P + R_A) の埋め込みをバックボーンを使用して計算します。
response_a = {k: v[:, 0, :] for k, v in inputs.items()}
embed_a = backbone(response_a)

# 2番目の応答: (P + R_B) の埋め込みを、同じバックボーンを使用して計算します。
response_b = {k: v[:, 1, :] for k, v in inputs.items()}
embed_b = backbone(response_b)

# 最終的な出力を計算します。
embeds = keras.layers.Concatenate(axis=-1)([embed_a, embed_b])
embeds = keras.layers.GlobalAveragePooling1D()(embeds)
outputs = keras.layers.Dense(3, activation="softmax", name="classifier")(embeds)
model = keras.Model(inputs, outputs)

# オプティマイザー、損失、メトリックとともにモデルをコンパイルします。
model.compile(
    optimizer=keras.optimizers.Adam(5e-6),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.02),
    metrics=[
        log_loss,
        keras.metrics.CategoricalAccuracy(name="accuracy"),
    ],
)
```

</div>
</details>

In [ ]:
# 入力層を定義します。
inputs = {
    "token_ids": keras.Input(shape=(2, None), dtype=tf.int32, name="token_ids"),
    "padding_mask": keras.Input(shape=(2, None), dtype=tf.int32, name="padding_mask"),
}
# DebertaV3Classifierバックボーンを作成します。
backbone = keras_nlp.models.DebertaV3Backbone.from_preset(
    CFG.preset,
)

# 最初の応答: (P + R_A) の埋め込みをバックボーンを使用して計算します。
response_a = {k: v[:, 0, :] for k, v in inputs.items()}
embed_a = backbone(response_a)

# 2番目の応答: (P + R_B) の埋め込みを、同じバックボーンを使用して計算します。
response_b = {k: v[:, 1, :] for k, v in inputs.items()}
embed_b = backbone(response_b)

# 最終的な出力を計算します。
embeds = keras.layers.Concatenate(axis=-1)([embed_a, embed_b])
embeds = keras.layers.GlobalAveragePooling1D()(embeds)
outputs = keras.layers.Dense(3, activation="softmax", name="classifier")(embeds)
model = keras.Model(inputs, outputs)

# オプティマイザー、損失、メトリックとともにモデルをコンパイルします。
model.compile(
    optimizer=keras.optimizers.Adam(5e-6),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.02),
    metrics=[
        log_loss,
        keras.metrics.CategoricalAccuracy(name="accuracy"),
    ],
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### Model Summary

</div>
<div class="column-right">

# 日本語訳

### モデルサマリー

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model.summary()
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルの概要を表示します。
model.summary()
```

</div>
</details>

In [ ]:
# モデルの概要を表示します。
model.summary()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### Model Plot

In the model graph below, it may seem there are **four** inputs, but actually, there are **two** as discussed before. Our input consists of two parts, one for each response. However, for each input, we have `token_ids` and `padding_mask`, which makes it look like we have four inputs, but in reality, we have two inputs.

</div>
<div class="column-right">

# 日本語訳

### モデルのプロット

以下のモデルグラフでは、**4つ**の入力があるように見えるかもしれませんが、実際には前述の通り**2つ**の入力があります。我々の入力は2つの部分から構成されており、それぞれの応答に対するものです。しかし、各入力には`token_ids`と`padding_mask`があるため、4つの入力があるように見えますが、実際には2つの入力となります。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Currently throwing error !! [probably library or env issue, so hopefully will be fixed soon]

# keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 現在エラーが発生しています!! [おそらくライブラリや環境の問題ですので、近いうちに修正されることを願っています]

# モデルのプロットを試みますが、エラーが発生する可能性があります。
# keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)
```

</div>
</details>

In [ ]:
# 現在エラーが発生しています!! [おそらくライブラリや環境の問題ですので、近いうちに修正されることを願っています]

# モデルのプロットを試みますが、エラーが発生する可能性があります。
# keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# 🚂 | Training

</div>
<div class="column-right">

# 日本語訳

# 🚂 | トレーニング

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Start training the model
history = model.fit(
    train_ds,
    epochs=CFG.epochs,
    validation_data=valid_ds,
    callbacks=[lr_cb, ckpt_cb]
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルのトレーニングを開始します。
history = model.fit(
    train_ds,
    epochs=CFG.epochs,
    validation_data=valid_ds,
    callbacks=[lr_cb, ckpt_cb]
)
```

</div>
</details>

In [ ]:
# モデルのトレーニングを開始します。
history = model.fit(
    train_ds,
    epochs=CFG.epochs,
    validation_data=valid_ds,
    callbacks=[lr_cb, ckpt_cb]
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Load Best Model

After training, let's load the weight with best result to get the best performance.

</div>
<div class="column-right">

# 日本語訳

## 最良モデルの読み込み

トレーニングが完了した後、最良の結果を得るために重みを読み込み、最良のパフォーマンスを得ましょう。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model.load_weights('/kaggle/working/best_model.weights.h5')
```

</div>
<div class="column-right">

# 日本語訳

```python
# 最良の重みを読み込みます。
model.load_weights('/kaggle/working/best_model.weights.h5')
```

</div>
</details>

In [ ]:
# 最良の重みを読み込みます。
model.load_weights('/kaggle/working/best_model.weights.h5')

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# 🧪 | Prediction

</div>
<div class="column-right">

# 日本語訳

# 🧪 | 予測

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Build test dataset
test_texts = test_df.options.tolist()
test_ds = build_dataset(test_texts,
                         batch_size=min(len(test_df), CFG.batch_size),
                         shuffle=False)
```

</div>
<div class="column-right">

# 日本語訳

```python
# テストデータセットを構築します。
test_texts = test_df.options.tolist()
test_ds = build_dataset(test_texts,
                         batch_size=min(len(test_df), CFG.batch_size),
                         shuffle=False)
```

</div>
</details>

In [ ]:
# テストデータセットを構築します。
test_texts = test_df.options.tolist()
test_ds = build_dataset(test_texts,
                         batch_size=min(len(test_df), CFG.batch_size),
                         shuffle=False)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Make predictions using the trained model on test data
test_preds = model.predict(test_ds, verbose=1)
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニングされたモデルを使用してテストデータに対して予測を行います。
test_preds = model.predict(test_ds, verbose=1)
```

</div>
</details>

In [ ]:
# トレーニングされたモデルを使用してテストデータに対して予測を行います。
test_preds = model.predict(test_ds, verbose=1)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# 📬 | Submission

Following code will prepare the submission file.

</div>
<div class="column-right">

# 日本語訳

# 📬 | 提出

以下のコードは、提出ファイルの準備をします。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
sub_df = test_df[["id"]].copy()
sub_df[CFG.class_names] = test_preds.tolist()
sub_df.to_csv("submission.csv", index=False)
sub_df.head()
```

</div>
<div class="column-right">

# 日本語訳

```python
# 提出用のデータフレームを作成します。
sub_df = test_df[["id"]].copy()
# 予測結果を追加します。
sub_df[CFG.class_names] = test_preds.tolist()
# 提出ファイルをCSV形式で保存します。
sub_df.to_csv("submission.csv", index=False)
# 最初の数行を表示します。
sub_df.head()
```

</div>
</details>

In [ ]:
# 提出用のデータフレームを作成します。
sub_df = test_df[["id"]].copy()
# 予測結果を追加します。
sub_df[CFG.class_names] = test_preds.tolist()
# 提出ファイルをCSV形式で保存します。
sub_df.to_csv("submission.csv", index=False)
# 最初の数行を表示します。
sub_df.head()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# 🔭 | Future Directions

In this notebook, we've achieved a good score with a small model and modest token length. But there's plenty of room to improve. Here's how:

1. Try bigger models like `Deberta-Base` or `Deberta-Small`, or even LLMs like `Gemma`.
2. Increase max token length to reduce loss of data.
3. Use a five-fold cross-validation and ensemble to make the model robust and get better scores.
4. Add augmentation like shuffling response orders for more robust performance.
5. Train for more epochs.
6. Tune the learning rate scheduler.

# 📌 | Reference

* [LLM Science Exam: KerasCore + KerasNLP [TPU]](https://www.kaggle.com/code/awsaf49/llm-science-exam-kerascore-kerasnlp-tpu)
* [AES 2.0: KerasNLP Starter](https://www.kaggle.com/code/awsaf49/aes-2-0-kerasnlp-starter)

</div>
<div class="column-right">

# 日本語訳

# 🔭 | 今後の方向性

このノートブックでは、小さなモデルと控えめなトークン長で良いスコアを達成しましたが、改善の余地はまだたくさんあります。以下の方法でさらなる向上を図ることができます：

1. `Deberta-Base`や`Deberta-Small`のような大きなモデル、あるいは`Gemma`のようなLLMを試してみる。
2. 最大トークン長を増やしてデータの損失を減らす。
3. 5フォールドクロスバリデーションとアンサンブルを使用してモデルを堅牢にし、より良いスコアを得る。
4. 応答の順序をシャッフルするなどの拡張を追加して、より堅牢なパフォーマンスを得る。
5. より多くのエポックでトレーニングする。
6. 学習率スケジューラーを調整する。

# 📌 | 参考文献

* [LLM Science Exam: KerasCore + KerasNLP [TPU]](https://www.kaggle.com/code/awsaf49/llm-science-exam-kerascore-kerasnlp-tpu)
* [AES 2.0: KerasNLP Starter](https://www.kaggle.com/code/awsaf49/aes-2-0-kerasnlp-starter)

</div>